# AuxTel almanac demo for {{ params.dayobs }}

With [Astroplan](https://astroplan.readthedocs.io/en/latest/index.html), a nightly report can include a basic almanac:

- sunset and sunrise times
- nautical twilight
- moon illumination
- moon altitude

In [ ]:
# Times Square parameters
dayobs = "20240108"

In [ ]:
from datetime import datetime, timedelta, timezone, UTC
from zoneinfo import ZoneInfo

from astroplan import Observer
from astropy.time import Time
from IPython.display import Markdown
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
local_timezone = ZoneInfo("America/Santiago")
timestamp_format = "%Y-%m-%d %H:%M:%S %Z"
auxtel_site = Observer.at_site("Rubin AuxTel")

dayobs_start = datetime.strptime(dayobs, '%Y%m%d').replace(
    hour=0, minute=0, second=0, tzinfo=timezone(-timedelta(hours=12))
)

# Datetimes to establish reference points for finding sunset times
# and measurement times for moon illumination
noon_local_datetime = dayobs_start.astimezone(local_timezone).replace(
    hour=12, minute=0, second=0, tzinfo=local_timezone
)
midnight_local_datetime = dayobs_start.astimezone(local_timezone).replace(
    hour=23, minute=59, second=59, tzinfo=local_timezone
)

# Astropy Time for astroplan calculations
noon_time = Time(noon_local_datetime, scale="utc")
midnight_time = Time(midnight_local_datetime, scale="utc")

In [ ]:
# Compute sunrise/sunset
sunset_datetime = auxtel_site.sun_set_time(
    noon_time,
    which="next"
).to_datetime(timezone=UTC)
sunrise_datetime = auxtel_site.sun_rise_time(
    noon_time,
    which="next"
).to_datetime(timezone=UTC)

# Nautical sunset and sunrise (preferred by observers over astronomical twilight)
evening_twilight_datetime = auxtel_site.twilight_evening_nautical(
    noon_time,
    which="next"
).to_datetime(timezone=UTC)
morning_twilight_datetime = auxtel_site.twilight_morning_nautical(
    noon_time,
    which="next"
).to_datetime(timezone=UTC)

In [ ]:
# Lunar almanac
moon_illumination = auxtel_site.moon_illumination(midnight_time)
moonrise_datetime = auxtel_site.moon_rise_time(
    Time(midnight_time),
    which="nearest"
).to_datetime(timezone=UTC)
moonset_datetime = auxtel_site.moon_set_time(
    Time(moonrise_datetime, scale="utc"),
    which="next"
).to_datetime(timezone=UTC)

## Sun set and rise

In [ ]:
template = f"""
| | Local time | UTC |
| :- | :- | :- |
| Sun set | {sunset_datetime.astimezone(local_timezone).strftime(timestamp_format)} | {sunset_datetime.strftime(timestamp_format)} |
| Evening twilight | {evening_twilight_datetime.astimezone(local_timezone).strftime(timestamp_format)} | {evening_twilight_datetime.strftime(timestamp_format)} |
| Morning twilight | {morning_twilight_datetime.astimezone(local_timezone).strftime(timestamp_format)} | {morning_twilight_datetime.strftime(timestamp_format)} |
| Sun rise | {sunrise_datetime.astimezone(local_timezone).strftime(timestamp_format)} | {sunrise_datetime.strftime(timestamp_format)} |

*Twilight is nautical (12 degrees below horizon).*
"""

Markdown(template)

In [ ]:
observing_hours = morning_twilight_datetime - evening_twilight_datetime
template = f"**Observing time:** {observing_hours / timedelta(hours=1.):.2f} hours"
Markdown(template)

## Moon

In [ ]:

def conf_axes(ax, label_xaxis=False, label_twilight=False):
    """Configure axis for plotting a night.

    Functionality includes:

    - plotting the twilight period (and optionally labelling it)
    - setting the axis limits
    - hiding the x-axis labels (optionally) as needed in the axes grid.
    """
    # Note: this function depends on global state for the sunset/sunrise
    # and twilight datetimes.
    ax.set_xlim([sunset_datetime, sunrise_datetime])
    ax.axvspan(sunset_datetime, evening_twilight_datetime, fc='r', alpha=0.2, zorder=-2)
    ax.axvspan(morning_twilight_datetime, sunrise_datetime, fc='r', alpha=0.2, zorder=-2)
    if not label_xaxis:
        ax.axes.xaxis.set_ticklabels([])

    if label_twilight:
        ax.text(
            evening_twilight_datetime - timedelta(minutes=30),
            0.5,
            "twilight",
            c='r',
            size=15,
            rotation=90,
            transform=ax.get_xaxis_transform(),
            horizontalalignment='center',
            verticalalignment='center',
            zorder=-1
        )
        ax.text(
            morning_twilight_datetime + timedelta(minutes=30),
            0.5,
            "twilight",
            c='r',
            size=15,
            rotation=90,
            transform=ax.get_xaxis_transform(),
            horizontalalignment='center',
            verticalalignment='center',
            zorder=-1
        )

_t1 = Time(sunset_datetime)
_t2 = Time(sunrise_datetime)
_t_grid = _t1 + (_t2 - _t1) * np.linspace(0., 1., 100)
moon_altaz = auxtel_site.moon_altaz(_t_grid)

_t = moon_altaz.obstime.to_datetime(timezone=UTC)
fig, ax = plt.subplots()
ax.plot(_t, moon_altaz.alt, lw=2)
ax.axhline(0., ls='--', c='k')
ax.set_xlabel("Time (MM-DD HH, UTC)")
ax.set_ylabel("Moon altitude (deg)")
ax.text(
    0.5,
    0.8,
    f"Moon illumination: {moon_illumination:.0%}",
    horizontalalignment='center',
    verticalalignment='top',
    transform=ax.transAxes,
    bbox=dict(facecolor='w', alpha=0.8)
)
ax.text(
    0.5,
    0.95,
    f"DAYOBS {dayobs}",
    horizontalalignment='center',
    verticalalignment='top',
    transform=ax.transAxes,
)
conf_axes(ax, label_xaxis=True, label_twilight=True)
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
plt.show()